### Yolo v3 다운로드 및 설치

In [ ]:
!git clone https://github.com/ultralytics/yolov3
!cd yolov3;pip install -qr requirements.txt

### kaggle kernel은 wandb 모듈이 이미 설치되어 있기 때문에 pip unistall 로 wandb 삭제 필요

In [ ]:
!pip uninstall -y wandb

In [ ]:
!wget -O incredibles.zip https://github.com/chulminkw/DLCV/blob/master/incredibles.zip?raw=true
!mkdir /kaggle/working/incredibles
!unzip incredibles.zip -d /kaggle/working/incredibles

### Dataset용 yaml 파일을 생성하고 학습 수행


In [ ]:
!wget -O /kaggle/working/incredibles/incredibles.yaml https://raw.githubusercontent.com/chulminkw/DLCV/master/data/util/incredibles.yaml

### kaggle 버전은 아래를 이용하여 incredibles.yaml을 kaggle kernel 환경에 맞도록 수정.

In [ ]:
!cp /kaggle/working/incredibles/incredibles.yaml /kaggle/working/incredibles/incredibles_temp.yaml
!sed 's/content/kaggle\/working/g' /kaggle/working/incredibles/incredibles_temp.yaml > /kaggle/working/incredibles/incredibles.yaml
'''반드시 아래 incredibles.yaml 에서 아래 train, val 설정이 /kaggle/working 디렉토리로 변경되었는지 확인
train: /kaggle/working/incredibles/images/train/ 
val: /kaggle/working/incredibles/images/train/
'''
!cat /kaggle/working/incredibles/incredibles.yaml

In [ ]:
'''
# kaggle kernel은 gdrive를 mount 할 수 없으므로 아래 명령어 수행하지 않고 /kaggle/working/ultra_workdir 로 생성. 
# Google Drive 접근을 위한 Mount 적용. 
import os, sys 
from google.colab import drive 

drive.mount('/content/gdrive')
# soft link로 Google Drive Directory 연결. 
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive
# Google Drive 밑에 Directory 생성. 이미 생성 되어 있을 시 오류 발생. 
'''
!mkdir -p "/kaggle/working/ultra_workdir"

In [ ]:
###  batch size는 8로 정할것. 16으로 설정시 성능이 좋아지지 않음. epoch는 150번으로 설정.    
!cd /kaggle/working/yolov3; python train.py --img 640 --batch 8 --epochs 150 --data /kaggle/working/incredibles/incredibles.yaml --weights yolov3.pt \
                                     --project=/kaggle/working/ultra_workdir --name incredibles --exist-ok 

### 단일 이미지 및 Video inference 수행 

In [ ]:
# image 파일 inference 
!cd /kaggle/working/yolov3;python detect.py --source /kaggle/working/incredibles/images/train/incredible_028.jpg \
                            --weights /kaggle/working/ultra_workdir/incredibles/weights/best.pt --conf 0.3 \
                            --project=/kaggle/working/data/output --name=run_image --exist-ok --line-thickness 2

In [ ]:
from IPython.display import Image, clear_output 

Image(filename='/kaggle/working/data/output/run_image/incredible_028.jpg', width=800)

In [ ]:
!wget -O /kaggle/working/incredibles/incredibles.mp4 https://github.com/chulminkw/DLCV/blob/master/data/video/incredibles.mp4?raw=true

In [ ]:
!cd /kaggle/working/yolov3;python detect.py --source /kaggle/working/incredibles/incredibles.mp4 \
                            --weights /kaggle/working/ultra_workdir/incredibles/weights/best.pt --conf 0.3 \
                            --project=/kaggle/working/data/output --name=run_image --exist-ok --line-thickness 2